# Walk through 3D lidar data
How does a self-driving car see the world? You can finally answer this question by walking through what a self-driving car sees. I downloaded the sample data from the AEV dataset, which you can find here: https://www.audi-electronics-venture.de/aev/web/de/driving-dataset/dataset.html. 
Download and extract the a2d2-preview folder in the same folder as this notebook :)

If this is too much code you can also download samples from other datasets, for example the KITTI dataset. 

In [1]:
import numpy as np
import numpy.linalg as la
import colorsys
import json
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

with open ('a2d2-preview/cams_lidars.json', 'r') as f:
    config = json.load(f)
    
EPSILON = 1.0e-10 # norm should not be small
def get_axes_of_a_view(view):
    x_axis = view['x-axis']
    y_axis = view['y-axis']
     
    x_axis_norm = la.norm(x_axis)
    y_axis_norm = la.norm(y_axis)
    
    if (x_axis_norm < EPSILON or y_axis_norm < EPSILON):
        raise ValueError("Norm of input vector(s) too small.")
        
    # normalize the axes
    x_axis = x_axis / x_axis_norm
    y_axis = y_axis / y_axis_norm
    
    # make a new y-axis which lies in the original x-y plane, but is orthogonal to x-axis
    y_axis = y_axis - x_axis * np.dot(y_axis, x_axis)
 
    # create orthogonal z-axis
    z_axis = np.cross(x_axis, y_axis)
    
    # calculate and check y-axis and z-axis norms
    y_axis_norm = la.norm(y_axis)
    z_axis_norm = la.norm(z_axis)
    
    if (y_axis_norm < EPSILON) or (z_axis_norm < EPSILON):
        raise ValueError("Norm of view axis vector(s) too small.")
        
    # make x/y/z-axes orthonormal
    y_axis = y_axis / y_axis_norm
    z_axis = z_axis / z_axis_norm
    
    return x_axis, y_axis, z_axis
def get_origin_of_a_view(view):
    return view['origin']

def get_transform_from_global(view):
    # get transform to global
    transform_to_global = get_transform_to_global(view)
    trans = np.eye(4)
    rot = np.transpose(transform_to_global[0:3, 0:3])
    trans[0:3, 0:3] = rot
    trans[0:3, 3] = np.dot(rot, -transform_to_global[0:3, 3])

    return trans

def get_transform_to_global(view):
    # get axes
    x_axis, y_axis, z_axis = get_axes_of_a_view(view)
    
    # get origin 
    origin = get_origin_of_a_view(view)
    transform_to_global = np.eye(4)
    
    # rotation
    transform_to_global[0:3, 0] = x_axis
    transform_to_global[0:3, 1] = y_axis
    transform_to_global[0:3, 2] = z_axis
    
    # origin
    transform_to_global[0:3, 3] = origin
    
    return transform_to_global

def transform_from_to(src, target):
    transform = np.dot(get_transform_from_global(target), \
                       get_transform_to_global(src))
    
    return transform
def project_lidar_from_to(lidar, src_view, target_view, key_points = 'points'):
    lidar = dict(lidar)
    trans = transform_from_to(src_view, target_view)
    points = lidar[key_points]
    points_hom = np.ones((points.shape[0], 4))
    points_hom[:, 0:3] = points
    points_trans = (np.dot(trans, points_hom.T)).T 
    lidar[key_points] = points_trans[:,0:3]
    
    return lidar



In [2]:

names = [("front_center", "frontcenter"), ("front_left", "frontleft"), ("front_right", "frontright"), ("side_left", "sideleft"), ("side_right", "sideright"), ("rear_center", "rearcenter")]
scans = list()
for foldername, lidarname in names:
    file_name_lidar = 'a2d2-preview/camera_lidar/20190401_145936/lidar/cam_'+foldername+'/20190401145936_lidar_'+lidarname+'_000017975.npz'
    lidar_front_center = np.load(file_name_lidar)
    vehicle_view = target_view = config['vehicle']['view']
    src_view_front_center = config['cameras'][foldername]['view']
    lidar_front_center = project_lidar_from_to(lidar_front_center,\
                                              src_view_front_center, \
                                              vehicle_view, key_points='pcloud_points')


    tosave = np.concatenate((lidar_front_center['pcloud_points'],np.expand_dims(lidar_front_center['pcloud_attr.reflectance'], axis=1)), axis=1)
    scan = np.array(tosave, dtype=np.float32)
    scan[:,:3] *= 0.25
    scan[:,3] /=255.0
    scans.append(scan)
    

In [3]:
tosave = np.concatenate(scans, axis=0)
tosave = tosave[:,[0,2,1,3]]
tosave.byteswap().tofile('../cars3.npy')

# Walk through semantically annotated data
In case you want to know what class a point belongs to: we can find this by projecting semantic segmentation onto the lidar points. 

In [4]:
file_name_lidar = 'a2d2-preview/camera_lidar_semantic/20181107_132730/lidar/cam_front_center/20181107132730_lidar_frontcenter_000005599.npz'
file_name_image = 'a2d2-preview/camera_lidar_semantic/20181107_132730/label/cam_front_center/20181107132730_label_frontcenter_000005599.png'
lidar_front_center = np.load(file_name_lidar)
vehicle_view = target_view = config['vehicle']['view']
src_view_front_center = config['cameras'][foldername]['view']
lidar_front_center = project_lidar_from_to(lidar_front_center,\
                                          src_view_front_center, \
                                          vehicle_view, key_points='points')


tosave = np.concatenate((lidar_front_center['points'],np.expand_dims(lidar_front_center['reflectance'], axis=1)), axis=1)
scan = np.array(tosave, dtype=np.float32)
scan[:,:3] *= 0.25
scan[:,3] /=255.0

image = plt.imread(file_name_image)
colors = list()
for row, col in zip(lidar_front_center['row'], lidar_front_center['col']):
    h, _, _ = colorsys.rgb_to_hsv(*image[int(row)][int(col)])
    colors.append(h)
    
scan[:,3] = np.array(colors)
scan = scan[:,[0,2,1,3]]
scan.byteswap().tofile('../cars4.npy')

# Add bounding boxes (premium feature)


In [5]:
import json


file_name_lidar = 'a2d2-preview/camera_lidar_semantic_bboxes/20180807_145028/lidar/cam_front_center/20180807145028_lidar_frontcenter_000028313.npz'
file_name_image = 'a2d2-preview/camera_lidar_semantic_bboxes/20180807_145028/camera/cam_front_center/20180807145028_camera_frontcenter_000028313.png'
file_name_label = 'a2d2-preview/camera_lidar_semantic_bboxes/20180807_145028/label3D/cam_front_center/20180807145028_label3D_frontcenter_000028313.json'


# file_name_lidar = 'a2d2-preview/camera_lidar_semantic_bboxes/20180810_142822/lidar/cam_front_center/20180810142822_lidar_frontcenter_000000426.npz'
# file_name_image = 'a2d2-preview/camera_lidar_semantic_bboxes/20180810_142822/camera/cam_front_center/20180810142822_camera_frontcenter_000000426.png'
# file_name_label = 'a2d2-preview/camera_lidar_semantic_bboxes/20180810_142822/label3D/cam_front_center/20180810142822_label3D_frontcenter_000000426.json'


lidar_front_center = np.load(file_name_lidar)
print(lidar_front_center['points'].shape)
vehicle_view = target_view = config['vehicle']['view']
src_view_front_center = config['cameras'][foldername]['view']

lidar_front_center = project_lidar_from_to(lidar_front_center,\
                                          src_view_front_center, \
                                          vehicle_view, key_points='points')

tosave = np.concatenate((lidar_front_center['points'],np.expand_dims(lidar_front_center['reflectance'], axis=1)), axis=1)
scan = np.array(tosave, dtype=np.float32)
scan[:,:3] *= 0.25
scan[:,3] /=255.0

print(scan.shape)
print(scan.dtype)
scan = scan[:,[0,2,1,3]]


scan.byteswap().tofile('../cars5.xyzi')

with open(file_name_label) as f:
    data = json.load(f)

concatenatedl = list()
for bbox in data: 
    # CX, CY, CZ, X, Y, Z, ROT_ANGLE
    concatenated = [*data[bbox]["center"], *data[bbox]["size"], data[bbox]["rot_angle"]]
    npar = np.array(concatenated)
    
    # Scale just like the point cloud
    npar[:6] *= 0.25
    
    # Switch Y and Z
    npar = npar[[0,2,1,3,5,4,6]]
    concatenatedl.append(npar)

# Save to a file
arr = np.array(concatenatedl, dtype=np.float32)
arr.byteswap().tofile('../cars5boxes.npy')

(10924, 3)
(10924, 4)
float32
